<a href="https://colab.research.google.com/github/aimhkimi74/Text-to-Image-Interior-Design-Generator/blob/main/livingroom_backend_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================
# STEP 1: Install Required Libraries
# ===============================
!pip install torch accelerate diffusers flask flask-cors pyngrok pillow --quiet

# ===============================
# STEP 2: Imports
# ===============================
import torch, io, base64
from flask import Flask, request, jsonify
from flask_cors import CORS
from diffusers import DiffusionPipeline
from PIL import Image
from pyngrok import ngrok

# ===============================
# STEP 3: Load Model + LoRA
# ===============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pipe = DiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
).to(device)

# Load your LoRA
pipe.load_lora_weights("aimhkimi74/sd15-lora-LivingRoom")
pipe.enable_attention_slicing()

# ===============================
# STEP 4: Setup Flask
# ===============================
app = Flask(__name__)
CORS(app)

@app.route("/health", methods=["GET"])
def health():
    return jsonify({"status": "ok", "device": str(device)}), 200

@app.route("/generate", methods=["POST"])
def generate():
    try:
        data = request.get_json(force=True)
        prompt = (data.get("prompt") or "").strip()
        if not prompt:
            return jsonify({"error": "No prompt provided"}), 400

        with torch.inference_mode():
            result = pipe(
                prompt,
                num_inference_steps=40,
                guidance_scale=7.5,
                height=512,
                width=512
            )
            image = result.images[0]

        buf = io.BytesIO()
        image.save(buf, format="JPEG", quality=95)
        img_b64 = base64.b64encode(buf.getvalue()).decode("utf-8")

        return jsonify({"image": img_b64}), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# ===============================
# STEP 5: Expose with ngrok + Run
# ===============================
ngrok.set_auth_token("YOUR_NGROK_TOKEN")  # 🔑 replace with your token
public_url = ngrok.connect(5000, bind_tls=True)
print(f"🌐 Public URL: {public_url}")

app.run(host="0.0.0.0", port=5000)
